## Installation of mne.connectivity package
If this is your first time to use this notebook, you will have to install ```mne.connectivity``` package library. In order to do so follow the following steps:
(in a command line window)
```
conda activate biosignals
conda install -c conda-forge mne-connectivity
```


# MNE Sample Dataset: Auditory and Visual Evoked Responses

This notebook uses the MNE-Python sample dataset, which contains MEG and EEG recordings from a single subject who was presented with auditory and visual stimuli.

## Dataset Details:
- **Subject**: A healthy adult volunteer
- **Recording**: 204 planar gradiometers, 102 magnetometers, and 60 EEG channels
- **Paradigm**: Simple auditory and visual stimulation with left/right conditions
- **Trials**: Approximately 100 epochs each for the different conditions

## Experimental Conditions in Detail:

### Auditory Experiment
The auditory experiment presents tone beeps to either the left or right ear:
- **Stimulus type**: Pure tone beeps
- **Frequency**: 1 kHz (1000 Hz)
- **Duration**: 50 ms
- **Intensity**: 60 dB sound pressure level
- **Presentation**: 
  - Left ear (Event code: '1' or 'Auditory/Left')
  - Right ear (Event code: '2' or 'Auditory/Right')
- **Inter-stimulus interval**: Variable, around 1-1.5 seconds
- **Neural response**: 
  - Both conditions activate the auditory cortices in the temporal lobes
  - Stronger contralateral activation (left ear stimuli activate right hemisphere more, and vice versa)
  - Peak responses typically occur around 100 ms (N100/M100) and 200 ms (P200/M200) post-stimulus

### Visual Experiment
The visual experiment involves presenting checkerboard patterns to the left and right visual fields separately:
- **Stimulus type**: Black and white checkerboard pattern
- **Presentation**: 
  - Left visual field (Event code: '3' or 'Visual/Left')
  - Right visual field (Event code: '4' or 'Visual/Right')
- **Duration**: 50 ms
- **Visual angle**: Approximately 4 degrees, presented about 5 degrees to the left or right of fixation
- **Contrast**: High contrast (black/white)
- **Inter-stimulus interval**: Variable, around 1-1.5 seconds
- **Neural response**: 
  - Left visual field stimuli primarily activate the right occipital cortex
  - Right visual field stimuli primarily activate the left occipital cortex
  - Characteristic components appear around 100 ms (P100/M100) and 170 ms (N170/M170) post-stimulus
  - This contralateral activation pattern allows for clear demonstration of the retinotopic organization of the visual system

## Research Applications:
This dataset is ideal for demonstrating:
- Basic preprocessing of MEG/EEG data
- Evoked response analysis 
- Source localization techniques
- Comparison between different sensor types
- Time-frequency analysis
- Contralateral organization of both auditory and visual systems
- Differences in spatial and temporal patterns between auditory and visual processing
- Hemispheric specialization and lateralization effects

The data follows the standard organization used in MNE-Python, making it easy to access events, 
raw recordings, and metadata through the MNE API.


In [23]:
%matplotlib qt

import mne
from mne.datasets import sample
from mne_connectivity import spectral_connectivity_epochs
from mne_connectivity.viz import plot_sensors_connectivity
import numpy as np
import matplotlib.pyplot as plt



In [24]:
# Load sample data
data_path = sample.data_path()
raw_fname = data_path / 'MEG' / 'sample' / 'sample_audvis_raw.fif'
event_fname = data_path / 'MEG' / 'sample' / 'sample_audvis_raw-eve.fif'

# Load raw EEG data
raw = mne.io.read_raw_fif(raw_fname, preload=True)
raw.pick_types(meg=False, eeg=True, stim=True)
raw.filter(1., 40., fir_design='firwin')



Opening raw data file /home/koutras/mne_data/MNE-sample-data/MEG/sample/sample_audvis_raw.fif...
    Read a total of 3 projection items:
        PCA-v1 (1 x 102)  idle
        PCA-v2 (1 x 102)  idle
        PCA-v3 (1 x 102)  idle


    Range : 25800 ... 192599 =     42.956 ...   320.670 secs
Ready.
Reading 0 ... 166799  =      0.000 ...   277.714 secs...
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 1983 samples (3.302 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.2s


<Raw | sample_audvis_raw.fif, 68 x 166800 (277.7 s), ~89.5 MiB, data loaded>

In [25]:
# Extract epochs for auditory events
events = mne.read_events(event_fname)
event_id = {'Auditory/Left': 1,
            'Auditory/Right': 2
            }
epochs = mne.Epochs(raw, events, event_id, tmin=-0.2, tmax=0.5, picks=['eeg'],baseline=(None,0), preload=True)



Not setting metadata
145 matching events found


Setting baseline interval to [-0.19979521315838786, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 145 events and 421 original time points ...
0 bad epochs dropped


In [26]:
fmin, fmax = 1, 20
sfreq = raw.info["sfreq"] # the sampling frequency
tmin = 0.0 # exclude the baseline period

# Compute connectivity using PLV
con_plv = spectral_connectivity_epochs(
    epochs['Auditory/Left'],
    method="plv",
    mode="multitaper",
    sfreq=sfreq,
    fmin=fmin,
    fmax=fmax,
    faverage=True,
    tmin=tmin,
    mt_adaptive=False,
    n_jobs=1,
)

# Explicitly request dense output format
# This should convert the data to a matrix shape (n_nodes x n_nodes)
try:
    # First, try the standard approach
    con_matrix = con_plv.get_data(output="dense")
    print("PLV matrix shape:", con_matrix.shape)
    
    # Plot the connectivity
    fig = plot_sensors_connectivity(epochs.info, con_matrix[:, :, 0])
    
except Exception as e:
    print(f"Error with dense output: {e}")
    
    # Alternative approach: manually create the dense matrix
    n_nodes = con_plv.n_nodes
    data = con_plv.get_data()
    
    # Create empty matrix
    con_matrix = np.zeros((n_nodes, n_nodes))
    
    # Generate all possible pairs
    import itertools
    pairs = list(itertools.combinations(range(n_nodes), 2))
    
    # Fill the matrix
    if len(pairs) == data.shape[0]:
        for idx, (i, j) in enumerate(pairs):
            value = data[idx, 0]  # Assuming data is shape (n_connections, 1)
            con_matrix[i, j] = value
            con_matrix[j, i] = value  # Make symmetric
        
        # Plot using the manually created matrix
        fig = plot_sensors_connectivity(epochs.info, con_matrix)
        plt.title('PLV Connectivity')
    else:
        print(f"Mismatch: {len(pairs)} pairs vs {data.shape[0]} connections")

Adding metadata with 3 columns
Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 1711 connections
    using t=0.000s..0.499s for estimation (301 points)
    frequencies: 2.0Hz..20.0Hz (10 points)
    connectivity scores will be averaged for each band
    Using multitaper spectrum estimation with 7 DPSS windows
    the following metrics will be computed: PLV
    computing cross-spectral density for epoch 1
    computing cross-spectral density for epoch 2
    computing cross-spectral density for epoch 3
    computing cross-spectral density for epoch 4
    computing cross-spectral density for epoch 5
    computing cross-spectral density for epoch 6
    computing cross-spectral density for epoch 7
    computing cross-spectral density for epoch 8
    computing cross-spectral density for epoch 9
    computing cross-spectral density for epoch 10


/tmp/ipykernel_193026/1023336071.py:6: RuntimeWarning: fmin=1.000 Hz corresponds to 0.501 < 5 cycles based on the epoch length 0.501 sec, need at least 5.000 sec epochs or fmin=9.977. Spectrum estimate will be unreliable.
  con_plv = spectral_connectivity_epochs(


    computing cross-spectral density for epoch 11
    computing cross-spectral density for epoch 12
    computing cross-spectral density for epoch 13
    computing cross-spectral density for epoch 14
    computing cross-spectral density for epoch 15
    computing cross-spectral density for epoch 16
    computing cross-spectral density for epoch 17
    computing cross-spectral density for epoch 18
    computing cross-spectral density for epoch 19
    computing cross-spectral density for epoch 20
    computing cross-spectral density for epoch 21
    computing cross-spectral density for epoch 22
    computing cross-spectral density for epoch 23
    computing cross-spectral density for epoch 24
    computing cross-spectral density for epoch 25
    computing cross-spectral density for epoch 26
    computing cross-spectral density for epoch 27
    computing cross-spectral density for epoch 28
    computing cross-spectral density for epoch 29
    computing cross-spectral density for epoch 30


In [27]:
# Check the data directly
plv_data = con_plv.get_data()
print("Raw PLV data shape:", plv_data.shape)
print("Min value:", plv_data.min())
print("Max value:", plv_data.max())
print("Mean value:", plv_data.mean())

# If values are extremely small, it might explain the error
if plv_data.max() < 1e-10:
    print("WARNING: PLV values are extremely small, possibly indicating no significant connections")

# Try a different plotting approach
con_matrix = con_plv.get_data(output="dense")
    



Raw PLV data shape: (3481, 1)
Min value: 0.0
Max value: 0.998940930492294
Mean value: 0.39320804119296293


In [28]:
# Get channel names from the epochs object
ch_names = epochs.info['ch_names']
n_channels = len(ch_names)

# Plot with channel names
plt.figure(figsize=(12, 10))

# Create the PLV matrix plot with channel names
im = plt.imshow(con_matrix[:, :, 0], cmap='viridis')
plt.colorbar(im, label='PLV')
plt.title('PLV Connectivity Matrix')

# Add channel names as tick labels
# If there are many channels, you might want to show only a subset
tick_spacing = max(1, n_channels // 15)  # Show at most ~15 labels to avoid crowding
tick_positions = np.arange(0, n_channels, tick_spacing)
tick_labels = [ch_names[i] for i in tick_positions]

plt.xticks(tick_positions, tick_labels, rotation=90)
plt.yticks(tick_positions, tick_labels)

plt.xlabel('EEG Channels')
plt.ylabel('EEG Channels')
plt.tight_layout()
plt.show()

In [29]:
raw.plot_sensors(show_names=True)
plt.show()

In [30]:
cn_right = con_matrix[:, :, 0]


In [31]:
cn_left = con_matrix[:, :, 0]

In [32]:
# Plot with channel names
plt.figure(figsize=(12, 10))

im2 = plt.imshow(cn_left-cn_right, cmap='viridis')
plt.colorbar(im2, label='PLV')
plt.title('PLV difference left-right Connectivity Matrix')

# Add channel names as tick labels
# If there are many channels, you might want to show only a subset
tick_spacing = max(1, n_channels // 15)  # Show at most ~15 labels to avoid crowding
tick_positions = np.arange(0, n_channels, tick_spacing)
tick_labels = [ch_names[i] for i in tick_positions]

plt.xticks(tick_positions, tick_labels, rotation=90)
plt.yticks(tick_positions, tick_labels)

plt.xlabel('EEG Channels')
plt.ylabel('EEG Channels')
plt.tight_layout()
plt.show()